In [ ]:
!pip install -r requirements.txt -q

In [ ]:
import os
from helpers.dataset import Conversation
from helpers.fair_forge import FairForge
from elasticsearch import Elasticsearch, helpers
from pydantic import BaseModel
import logging

In [ ]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
agentic_index = f"{dataset}-agentic"

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
class AgenticBatch(BaseModel):
    agentic_leviathan_type: str
    agentic_leviathan_ground_truth: dict
    agentic_leviathan: dict
    session_id: str
    assistant_id: str
    qa_id: str

In [ ]:
class AgenticAnalyzer(FairForge):
    def _check_tool(self, data):
        if type(data) is dict and 'name' in data and 'parameters' in data:
            return True
        else:
            return False
            
    def process(self, thread: Conversation):
        for batch in thread.conversation:
            leviathan_keys = batch.leviathan.keys()
            for key in batch.ground_truth_leviathan.keys():
                if key in leviathan_keys:
                    batch = AgenticBatch(
                        agentic_leviathan_type=key,
                        agentic_leviathan_ground_truth = batch.ground_truth_leviathan[key],
                        agentic_leviathan = batch.leviathan[key],
                        session_id=thread.session_id,
                        assistant_id=thread.assistant_id,
                        qa_id=batch.qa_id
                    )
                    self.metrics.append(batch)
        logging.info(f"Finished processing thread for session_id: {thread.session_id}")

In [ ]:
agentic = AgenticAnalyzer()
metrics = agentic.pipeline()

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
mapping_agentic = {
  "mappings": {
    "properties": {
      "agentic_leviathan_type": {"type": "text"},
      "agentic_leviathan_ground_truth": {"type": "object"},
      "agentic_leviathan": {"type": "object"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type":"keyword"}
    }
  }
}

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
recreate_index(agentic_index, mapping_agentic)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": agentic_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")